# LISTA 1

## GLOBAL SETUP

In [1]:
import numpy as np
import os
import statistics
from tqdm.notebook import tqdm
import re
from sys import getsizeof
import sys
import copy
import nltk
import collections
import random
import itertools

## ZADANIE 1

In [38]:
def Succ1Grams():
    data = set()
    
    #num_lines = sum(1 for line in open('poleval_2grams.txt'))
    num_lines = 59134224
    lines_read = 0
    with open("poleval_2grams.txt", "r") as f:
        with tqdm(total=num_lines) as pbar:
            line = f.readline()
            while line:
                if lines_read > 10000:
                    pbar.update(lines_read)
                    lines_read = 0
                    pbar.set_description(f"memory used {sys.getsizeof(data)/1024/1024} MB")
                lines_read += 1
                wc, wl, wp = line.replace("\n", "").split(" ")
                data.add(wl)
                data.add(wp)

                #if len(t) < 50:
                #    data.append(t)
                line = f.readline()
                if len(data) > 10000000:
                    break
    return data

singletons = Succ1Grams()
maxlen = max(len(s) for s in singletons)
print(f"read {len(singletons)}, longest one is {maxlen}")
maxlen = 25


read 3591116, longest one is 125


In [3]:
def OwnMatch(w):
    graph = [[] for _ in w]
    for slit in range(len(w)):
        for forward_succ in range(5):
            taken = w[slit:slit+forward_succ+1]
            if len(taken) == forward_succ+1:
                if taken in singletons:
                    graph[slit].append(slit + forward_succ + 1)

    #print(f"neighbor matrix")
    #display(graph)

    dp = [(0, None) for _ in range(len(w)+1)]
    for i in range(len(w)-1):
        for jt in graph[i]:
            if dp[jt][0] < (dp[i][0] + (i-jt)**2):
                dp[jt] = (dp[i][0] + (i-jt)**2, i)

    #display(dp)

    baked = []
    ls = dp[-1][1]
    fs = len(w)
    while ls != None:
        baked.append(w[ls:fs])
        fs = ls
        ls = dp[ls][1]
        
    baked.reverse()
    
    return baked 

In [4]:
def MaxMatch(w, verbose = False):
    wp = w
    tokenized = []
    succ = 1
    best = (0, "reeee")
    
    while len(w) > 0:
        try_ = w[:succ]
        if try_ in singletons:
            if verbose:
                print(f"found candidate {try_}")
            best = (succ, try_) 

        succ += 1
        
        if succ > maxlen or try_ == w:
            if best[0] == 0:
                raise Exception(f"could not match {w[:succ]} in {wp}")
            else:
                if verbose:
                    print(f"longest match was {best[1]}")
                tokenized.append(best[1])
                if verbose:
                    print(f"transforming {w} -> {w[best[0]:]}")
                w = w[best[0]:]
                succ = 1
                best = (0, "reeee")
    return tokenized

In [5]:
# implementation stolen from https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/
# since jupyter considered that importing
# python-levenstein is on the same level as breaking the geneva
# convention

def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

In [6]:
def perform_test(compare_string):
    digested = compare_string.replace(" ", "")
    p1 = " ".join(MaxMatch(digested))
    p2 = " ".join(OwnMatch(digested))
    s1 = levenshteinDistanceDP(compare_string, p1)
    s2 = levenshteinDistanceDP(compare_string, p2)
    #print(compare_string)
    #print(f"MaxMatch returned {p1}, score {s1}")
    #print(f"OwnCrap returned {p2}, score {s2}")
    return (s1, s2)

In [7]:
poem = '''Jedzie Wojtuś popod lasem, skrajem dąbrowy,
Wiezie se do domu laser, laser gazowy.
Pytali się go kolesie, po co to nabył?
- A bo właśnie stał w GS-ie, więc kupiłem go Teresie.
Dyć coś zawsze przywieźć chce się dla swej baby!
Jedzie Wojtuś popod lasem, po twardym dukcie.
Wiezie se do domu laser, czyta instrukcję.
A instrukcja jest ciekawa: Włączyć do prądu,
To ten laser wszystko skrawa, bez różnicy, stal czy trawa
I w ogóle jest zabawa prawie bez swądu.
Jedzie Wojtuś popod lasem, woła: Wio, wiśta!
Wiezie se do domu laser, tak se rozmyśla:
Jak w tym dojdę do biegłości, będzie wygodnie,
Jest tu paru wrednych gości, przyceluję se w skrytości
I padalcom z odległości podpalę spodnie!
Jedzie Wojtuś popod lasem, rad, że o rany
Wiezie se do domu laser, układa plany:
Niech spróbuje na rowerze jeździć Kaczmarski,
Zaraz w dętkę mu przymierzę, i kartofle się obierze,
I wywierci dziury w serze, by był śwajcarski!
Jedzie Wojtuś popod lasem, skręcił przy POM-ie.
Wiezie se do domu laser schowany w słomie.
Na podwórku cielę bryka, kaczki na stawie...
Wdziera, wdziera się technika coraz częściej w dom rolnika,
Jeszcze to nie Ameryka.... Ale już prawie!!! '''

tMM, tMC = [], []
for l in tqdm(poem.split("\n")):
    retMM, retMC = perform_test(l.lower().replace(",", ""))
    tMM.append(retMM)
    tMC.append(retMC)
    
print("MaxMatch:")
print(f"Mean: {statistics.mean(tMM)}")
print(f"Stdev: {statistics.stdev(tMM)}")
print(f"max: {max(tMM)}")
print(f"min: {min(tMM)}")

print("MyCrap:")
print(f"Mean: {statistics.mean(tMC)}")
print(f"Stdev: {statistics.stdev(tMC)}")
print(f"max: {max(tMC)}")
print(f"min: {min(tMC)}")


#MaxMatch:
#Mean: 5.96
#Stdev: 4.6141087980237305
#max: 17.0
#min: 0.0
#MyCrap:
#Mean: 13.68
#Stdev: 12.294036494712927
#max: 48.0
#min: 4.0


MaxMatch:
Mean: 5.6
Stdev: 3.547299442298794
max: 12.0
min: 0.0
MyCrap:
Mean: 17.8
Stdev: 16.91646141090585
max: 48.0
min: 3.0


## ZADANIE 2 i 3


In [2]:
def calculate_bigrams():
    ins = collections.defaultdict(lambda: 0)
    bigrams = collections.defaultdict(lambda: copy.deepcopy(ins))
    num_lines = 59134224
    lines_read = 0
    maxim = 0
    with open("poleval_2grams.txt", "r") as f:
        with tqdm(total=num_lines) as pbar:
            line = f.readline()
            while line:
                if lines_read > 10000:
                    pbar.update(lines_read)
                    lines_read = 0
                    pbar.set_description(f"memory used {sys.getsizeof(bigrams)/1024/1024} MB")
                lines_read += 1
                wc, wl, wp = line.replace("\n", "").split(" ")
                if int(wc) > 2:
                    bigrams[wl][wp] += int(wc)
                    maxim = max(bigrams[wl][wp], maxim)

                line = f.readline()
                if len(bigrams) > 10000000:
                    break
    
    for w1 in bigrams.keys():
        for w2 in bigrams[w1].keys():
            bigrams[w1][w2] = float(bigrams[w1][w2])/float(maxim)
    return bigrams
            
bigrams = calculate_bigrams()
bigrams['dupa']

defaultdict(<function __main__.calculate_bigrams.<locals>.<lambda>()>,
            {'.': 1.2985921145254358e-06,
             ',': 1.8757441654256295e-06,
             '-': 1.923840169667312e-07,
             'w': 5.290560466585109e-07,
             'z': 7.695360678669248e-07,
             'i': 4.80960042416828e-07,
             'blada': 9.61920084833656e-07,
             'a': 1.923840169667312e-07,
             'latynoski': 2.40480021208414e-07,
             'nie': 3.3667202969177965e-07,
             'zbita': 4.3286403817514525e-07,
             'dupy': 1.923840169667312e-07,
             'boli': 1.923840169667312e-07,
             'sex': 3.3667202969177965e-07,
             'się': 1.4428801272504842e-07,
             'to': 1.4428801272504842e-07,
             'ze': 2.8857602545009683e-07,
             'weza': 1.4428801272504842e-07,
             'jest': 2.8857602545009683e-07,
             '...': 3.3667202969177965e-07,
             'na': 2.40480021208414e-07,
             'albo': 1

In [3]:
import time
def generate_story(max_len, starting, distribution):
    if max_len == 0:
        max_len = 1000000
    if starting == None:
        starting = "<BOS>"
        
    while(max_len > 0):
        print(starting, end = ' ')
        max_len -= 1
        candidates = bigrams[starting]
        
        if distribution == "random":
            candidates = [i[0] for i in candidates.items()]
            if len(candidates) == 0:
                return
            starting = random.choice(candidates) 
            
        if distribution == "weight":
            if len([i[0] for i in candidates.items()]) == 0:
                return
            
            all_ents = sum([i[1] for i in candidates.items()])
            chosen = random.randint(0, all_ents)
            for cit in candidates.items():
                chosen -= cit[1]
                if chosen <= 0:
                    starting = cit[0]
                    break
                    
            

In [4]:
for i in range(10):
    generate_story(0, "papież", "weight")
    print("\n=======")

papież jest głosowanie , nakazów zapłaty za wykonywanie tych słów ) . <EOS> 
papież jan piotr jest to pielęgniarki i środowiskowe , a następnie o zmianie niektórych innych źródeł , czyli nie pojawiła się w związku z dnia 31 marca 2004 r. <EOS> 
papież w dramacie telewizyjnym , woda nie tylko w klimacie tropikalnym i przekształcić się na pomocy na liście : świnoujście sa w polsce od przedmiotów : dlaczego otrzymał nagrodę , jak najszybsze ich wpływem wilgoci i onkologii i czystości , który w którym jest własnością . <EOS> 
papież bonifacy ix . <EOS> 
papież urban , sprzeciwiał się największą , stefana żeromskiego ) lub dodatków odzieżowych i najważniejszych festiwalach : nie powinno być przez żydów z liczbą emerytów i społecznych , maria nuova , natomiast dynamika w czwórce podwójnej na niej dwa dni otwarte w 103 . <EOS> 
papież klemens v w poprzedniej części przed ewentualnymi zmianami , zdobył drużynowe mistrzostwo świata , tylko i zatrudnienia , wszystkie inne środowiska i 2007 ) , p

## ZADANIE 4

In [3]:
def calculate_sufix(suflen):
    ins = collections.defaultdict(lambda: 0)
    sufgrams = collections.defaultdict(lambda: copy.deepcopy(ins))
    procd = 0
    pbar = tqdm(total=len(bigrams.keys()))
    
    for w1 in bigrams.keys():
        procd += 1

        if procd > 1000:
            procd = 0
            pbar.update(1000)
            pbar.set_description(f"memory used {str(sys.getsizeof(sufgrams)/1024/1024)[:5]} MB")

        if len(w1) > suflen:
            pref_sufix = str(w1)[-suflen:]
            for w2 in bigrams[w1].keys():
                if len(str(w2)) > suflen:
                    suf_sufix = str(w2)[-suflen:]
                    sufgrams[pref_sufix][suf_sufix] += bigrams[w1][w2]
                    
    # normalize ...
    biggest = 0
    for w1 in sufgrams.keys():
        for w2 in sufgrams[w1].keys():
            biggest = max(biggest, sufgrams[w1][w2])
    print(f"maximum score was {biggest}")
    for w1 in sufgrams.keys():
        for w2 in sufgrams[w1].keys():
            sufgrams[w1][w2] = float(sufgrams[w1][w2])/float(biggest)
    
    pbar.close()
    return sufgrams

In [4]:
bigram_suflist = [calculate_sufix(i) for i in range(1, 6)]

maximum score was 0.30078178130967553



maximum score was 0.09357313295628578



maximum score was 0.04880373694410105



maximum score was 0.012760014213331365



maximum score was 0.00618716617765856



In [5]:
def succ_3grams():
    ins = collections.defaultdict(lambda: 0)
    insd = collections.defaultdict(lambda: copy.deepcopy(ins))
    trigrams = collections.defaultdict(lambda: copy.deepcopy(insd))
    
    num_lines = 179473348
    lines_read = 0
    mv = 0
    with open("poleval_3grams.txt", "r") as f:
        with tqdm(total=num_lines) as pbar:
            line = f.readline()
            while line:
                if lines_read > 10000:
                    pbar.update(lines_read)
                    lines_read = 0
                    pbar.set_description(f"memory used {sys.getsizeof(trigrams)/1024/1024} MB")

                lines_read += 1
                try:
                    wc, w1, w2, w3 = line.replace("\n", "").split(" ")
                    if int(wc) > 1:
                        trigrams[w1][w2][w3] += int(wc)
                        mv = max(mv, trigrams[w1][w2][w3])
                except Exception:
                    pass
                line = f.readline()
 
    return (trigrams, mv)
            
trigrams, max_c = succ_3grams()
print(f"normalize by {max_c}")
for w1 in trigrams.keys():
    for w2 in trigrams[w1].keys():
        for w3 in trigrams[w1][w2].keys():
            trigrams[w1][w2][w3] = float(trigrams[w1][w2][w3])/float(max_c)
trigrams['dupa']


normalize by 762373


defaultdict(<function __main__.succ_3grams.<locals>.<lambda>()>,
            {',': defaultdict(<function __main__.succ_3grams.<locals>.<lambda>()>,
                         {'tak': 3.9350816463856934e-06,
                          'bo': 3.9350816463856934e-06,
                          'nie': 3.9350816463856934e-06}),
             'przebieralnia': defaultdict(<function __main__.succ_3grams.<locals>.<lambda>()>,
                         {'na': 2.6233877642571285e-06}),
             'orgasms': defaultdict(<function __main__.succ_3grams.<locals>.<lambda>()>,
                         {'porno': 2.6233877642571285e-06}),
             'i': defaultdict(<function __main__.succ_3grams.<locals>.<lambda>()>,
                         {'kamieni': 3.9350816463856934e-06}),
             '...': defaultdict(<function __main__.succ_3grams.<locals>.<lambda>()>,
                         {'<EOS>': 7.870163292771387e-06}),
             'w': defaultdict(<function __main__.succ_3grams.<locals>.<lambda>()>,
   

In [6]:
def calculate_sufix_3gram(suflen):
    ins = collections.defaultdict(lambda: 0)
    insd = collections.defaultdict(lambda: copy.deepcopy(ins))
    sufgrams = collections.defaultdict(lambda: copy.deepcopy(insd))
    procd = 0
    pbar = tqdm(total=len(trigrams.keys()))
    
    for w1 in trigrams.keys():
        procd += 1

        if procd > 1000:
            procd = 0
            pbar.update(1000)
            pbar.set_description(f"memory used {str(sys.getsizeof(sufgrams)/1024/1024)[:5]} MB")

        if len(w1) > suflen:
            pref_sufix = str(w1)[-suflen:]
            for w2 in trigrams[w1].keys():
                if len(str(w2)) > suflen:
                    suf_sufix = str(w2)[-suflen:]
                    for w3 in trigrams[w1][w2].keys():
                        if len(str(w3)) > suflen:
                            suf_suf_sufix = str(w3)[-suflen:]
                            sufgrams[pref_sufix][suf_sufix][suf_suf_sufix] += trigrams[w1][w2][w3]
                    
    # normalize ...
    biggest = 0
    for w1 in sufgrams.keys():
        for w2 in sufgrams[w1].keys():
            for w3 in sufgrams[w1][w2].keys():
                biggest = max(biggest, sufgrams[w1][w2][w3])
    print(f"maximum score was {biggest}")
    for w1 in sufgrams.keys():
        for w2 in sufgrams[w1].keys():
            for w3 in sufgrams[w1][w2].keys():
                sufgrams[w1][w2][w3] = float(sufgrams[w1][w2][w3])/float(biggest)
    
    pbar.close()
    return sufgrams

In [7]:
trigram_suflist = [calculate_sufix_3gram(i) for i in range(1, 6)]

maximum score was 1.1575646042039072



maximum score was 0.2358399366188816



maximum score was 0.0691183974248839



maximum score was 0.03963807742404309



maximum score was 0.03703961184354644



In [47]:
def calculate_score(results, desired):
    for resit, res in enumerate(results):
        if desired == " ".join(res[1:-1]):
            return 1.0/(resit + 1.0)
    return 0.0

In [49]:
def check_for_bigram(wa, wb):
    return bigrams[wa][wb]

def check_for_bisuf(wa, wb):
    for it in range(5, 1, -1):
        best = 0
        if len(wa) > it and len(wb) > it:
            sufa = wa[-it:]
            sufb = wb[-it:]
            if bigram_suflist[it-1][sufa][sufb] > 0.0:
                return (bigram_suflist[it-1][sufa][sufb], it)

def check_for_trigram(wa, wb, wc):
    if trigrams[wa][wb][wc] > 0:
        return trigrams[wa][wb][wc]
    return 0

def check_for_trisuf(wa, wb, wc):
    for it in range(5, 1, -1):
        best = 0
        if len(wa) > it and len(wb) > it:
            sufa = wa[-it:]
            sufb = wb[-it:]
            sufc = wc[-it:]
            if trigram_suflist[it-1][sufa][sufb][sufc] > 0.0:
                return (trigram_suflist[it-1][sufa][sufb][sufc], it)


def permutation_analizer(sentence, normalization = None):
    results = []
    norm_vals = [0.0, 0.0, 0.0]
    for pp in itertools.permutations(sentence):
        p = ["<BOS>"] + list(pp) + ["<EOS>"]
        score = 1.0
        taintcount = []
        for pit in range(len(p) - 1):
            # check for full trigram
            chunkscore = 0.0
            if pit < len(p) - 2:
                a1 = check_for_trigram(p[pit], p[pit+1], p[pit+2])
            # check for full bigram
            if pit < (len(p) - 1):
                a2 = check_for_bigram(p[pit], p[pit+1])
            
            # check for prefsizes.
            if pit < len(p) - 1:
                discore = check_for_bisuf(p[pit], p[pit+1])
            else:
                discore = None

            if pit < len(p) - 2:
                triscore = check_for_trisuf(p[pit], p[pit+1], p[pit+2])
            else:
                triscore = None

            if discore == None and triscore == None:
                a3 = 0.0
            elif discore == None and triscore is not None:
                a3 = triscore[0]
            elif discore is not None and triscore == None:
                a3 = discore[0]
            else:
                if discore[1] > triscore[1]:
                    a3 = discore[0]
                else:
                    a3 = triscore[1]
                
            if a1 + a2 + a3 == 0.0:
                taintcount.append(p[pit])
                chunkscore = 1e-100
            else: 
                if normalization is None:
                    chunkscore = a1+a2+a3
                else:
                    chunkscore = a1*normalization[0] + a2*normalization[1] + a3*normalization[2]
                    
            score *= chunkscore
            norm_vals = [
                max(norm_vals[vit], vi) for vit, vi in enumerate([a1, a2, a3])
            ]
        
        results.append((-score, p, taintcount))
    
    if normalization is not None:
        res = sorted(results)
        return res
        for r in res[:10]:
            print(r)
    else:
        print("that was the normalization pass, please rerun with this argument")

    return norm_vals
    
pres = permutation_analizer("wczoraj wieczorem spotkałem pewną piękną kobietę".split(" "), 
                     [1.0,
                      0.001,
                      0.000001,
                     ])

calculate_score(
    [i[1] for i in pres],
    "wczoraj wieczorem spotkałem pewną piękną kobietę")
#permutation_analizer(["judyta", "dała", "wczoraj", "stefanowi", "czekoladki"])

0.0030959752321981426

In [71]:
rundict = {}

In [79]:
def run_test_suite(norm = [1.0, 0.0, 0.0]):
    with open("test.txt", "r") as f:
        results = []
        for t in f.read().split("\n"):
            test = t.split(" ")
            response = permutation_analizer(test, norm)
            results.append(calculate_score([r[1] for r in response], t))
    return (statistics.mean(results), statistics.stdev(results))

norms = []
# 1000000.0, 1000000000.0
for i in [1000000000.0,100000000000.0, 1000000000000.0, 10000000000000.0, 1000000000000000.0]:
    for j in [1000000000.0,100000000000.0, 1000000000000.0, 10000000000000.0, 1000000000000000.0]:
        for k in [1.0]:
            norms.append([i, j, k])

blessed = []
for norm in tqdm(norms):
    if f"{norm[0]},{norm[1]},{norm[2]}" in rundict:
        blessed.append((rundict[f"{norm[0]},{norm[1]},{norm[2]}"], norm))
    else:
        print(f"dry run for {norm}")
        run = run_test_suite(norm)
        rundict[f"{norm[0]},{norm[1]},{norm[2]}"] = run
        blessed.append((run, norm))

dry run for [1000000000.0, 100000000000.0, 1.0]
dry run for [1000000000.0, 10000000000000.0, 1.0]
dry run for [100000000000.0, 1000000000.0, 1.0]
dry run for [100000000000.0, 100000000000.0, 1.0]
dry run for [100000000000.0, 1000000000000.0, 1.0]
dry run for [100000000000.0, 10000000000000.0, 1.0]
dry run for [100000000000.0, 1000000000000000.0, 1.0]
dry run for [1000000000000.0, 100000000000.0, 1.0]
dry run for [1000000000000.0, 10000000000000.0, 1.0]
dry run for [10000000000000.0, 1000000000.0, 1.0]
dry run for [10000000000000.0, 100000000000.0, 1.0]
dry run for [10000000000000.0, 1000000000000.0, 1.0]
dry run for [10000000000000.0, 10000000000000.0, 1.0]
dry run for [10000000000000.0, 1000000000000000.0, 1.0]
dry run for [1000000000000000.0, 100000000000.0, 1.0]
dry run for [1000000000000000.0, 10000000000000.0, 1.0]



In [80]:
display(sorted(blessed)[-5:])

[((0.28979054852981806, 0.37192165717677056),
  [100000000000.0, 1000000000000.0, 1.0]),
 ((0.29278514683329354, 0.35893466101620186),
  [1000000000.0, 1000000000.0, 1.0]),
 ((0.29507451196643275, 0.35394991544017557),
  [1000000000000000.0, 100000000000.0, 1.0]),
 ((0.30439954407513237, 0.36364063543562714),
  [1000000000000.0, 1000000000.0, 1.0]),
 ((0.30780482954995037, 0.36285218329853186),
  [10000000000000.0, 1000000000.0, 1.0])]